# Visualizing the types of supports used by mobile carriers in France  

*Charles Laverdure - 26 July 2023*

## Introduction 


## Librairies 

In [16]:
%reset
import pandas as pd 
import numpy as np
import geopandas as gpd
import glob
import os
# import geodatasets
import contextily as cx
from PIL import Image, ImageDraw

## Variables 

In [17]:
correspondance_nature_support = pd.read_table('data/anfr/SUP_NATURE.txt', sep=';')
df_aggregated = pd.read_csv('data/anfr/20230720150852_observatoire_2g_3g_4g.csv', sep=';')
display(df_aggregated.head(2))

,id,adm_lb_nom,sup_id,emr_lb_systeme,emr_dt,sta_nm_dpt,code_insee,generation,date_maj,sta_nm_anfr,...,tpo_id,adr_lb_lieu,adr_lb_add1,adr_lb_add2,adr_lb_add3,adr_nm_cp,com_cd_insee,coordonnees,coord,statut
0,1,SFR,513700,LTE 1800,2016-12-20,031,31118,4G,2023-07-19,0312700486,...,65.0,LES HEBRAILS,RTE DE BOULOC,NaN,NaN,31620,31118,"43.793333333333, 1.375555555556",43°47'36''N 1°22'32''E,En service
1,2,SFR,513700,GSM 900,2015-12-09,031,31118,2G,2023-07-19,0312700486,...,65.0,LES HEBRAILS,RTE DE BOULOC,NaN,NaN,31620,31118,"43.793333333333, 1.375555555556",43°47'36''N 1°22'32''E,En service


In [18]:
df_major_ops = df_aggregated[df_aggregated['adm_lb_nom'].isin(['ORANGE', 'SFR', 'FREE MOBILE', 'BOUYGUES TELECOM'])]
df_major_ops = df_major_ops[df_major_ops['statut'] == 'En service']
df_major_ops = df_major_ops.drop(['id', 'sta_nm_dpt', 'date_maj', 'tpo_id', 'adr_lb_lieu', 'adr_lb_add1', 'adr_lb_add2', 'generation', 'adr_lb_add3', 'adr_nm_cp', 'com_cd_insee', 'coord', 'statut'], axis = 1)
# discard rows where value of code_insee starts with 97
df_major_ops = df_major_ops[~df_major_ops['code_insee'].str.startswith('97')]
# transform coordonnees column into two columns (latitude and longitude)
df_major_ops[['latitude', 'longitude']] = df_major_ops['coordonnees'].str.split(',', expand=True)
df_major_ops = df_major_ops.drop(['coordonnees'], axis = 1)
df_major_ops['emr_dt'] = pd.to_datetime(df_major_ops['emr_dt'], format='%Y-%m-%d')
df_grouped = df_major_ops.groupby('sta_nm_anfr').agg({'sup_id': 'first', 'adm_lb_nom' : 'first', 'emr_lb_systeme': lambda x: set(list(x)), 'emr_dt': lambda x: min(x), 'code_insee': 'first', 'nat_id': 'first', 'sup_nm_haut': 'first', 'latitude': 'first', 'longitude': 'first'})
display(df_grouped.head(2))
display(df_grouped.shape)

,sup_id,adm_lb_nom,emr_lb_systeme,emr_dt,code_insee,nat_id,sup_nm_haut,latitude,longitude
sta_nm_anfr,,,,,,,,,
0010990001,560273,FREE MOBILE,"{LTE 1800, LTE 700, LTE 2600, UMTS 900, LTE 2100}",2011-08-26,01344,33,89.0,46.206944444444,5.206666666667
0010990002,891727,FREE MOBILE,"{LTE 700, LTE 2600, LTE 2100, UMTS 900, LTE 1800}",2012-12-12,01053,38,27.5,46.202222222222,5.243055555556


(106232, 9)

In [19]:
# rename the NAT_ID column in correspondance_nature_support to nat_id
df_1 = correspondance_nature_support.rename(columns={'NAT_ID': 'nat_id'})
# join the two dataframes on nat_id and NAT_ID
df = df_grouped.merge(df_1, on='nat_id', how='left')
df = df.drop(columns=['nat_id'])


In [30]:
def display_value_counts(df_for_count):
    value_count = df_for_count['NAT_LB_NOM'].value_counts(dropna=False)
    display(value_count)

In [34]:
def display_value_count_per_op(df_spe):
    for op in ['ORANGE', 'SFR', 'FREE MOBILE', 'BOUYGUES TELECOM']:
        print(op)
        df_operateur = df_spe[df_spe['adm_lb_nom'] == op]
        display_value_counts(df_operateur)
        display("length: {}".format(len(df_operateur)))
        print('\n\n')
        print('------------------')

In [35]:
paris_data = df[df['code_insee'].str.startswith('75')]
display_value_count_per_op(paris_data)

ORANGE


Immeuble                                                                              341
Bâtiment                                                                              187
Intérieur sous-terrain                                                                156
Tunnel                                                                                 29
Pylône tubulaire                                                                        3
Ouvrage d'art (pont, viaduc)                                                            3
Monument religieux                                                                      2
Pylône autostable                                                                       2
Monument historique                                                                     2
Intérieur galerie                                                                       2
Ouvrage de signalisation (portique routier, panneau routier, panneau publicitaire)      1
Pylône hau

'length: 729'




------------------
SFR


Immeuble                                                                              331
Intérieur sous-terrain                                                                162
Bâtiment                                                                               99
Tunnel                                                                                 30
Ouvrage de signalisation (portique routier, panneau routier, panneau publicitaire)     13
Ouvrage d'art (pont, viaduc)                                                            6
Monument historique                                                                     3
Pylône tubulaire                                                                        2
Mât béton                                                                               2
Pylône autostable                                                                       1
Mât métallique                                                                          1
Monument r

...

'length: 677'




------------------
BOUYGUES TELECOM


Immeuble                        521
Bâtiment                        101
Intérieur sous-terrain           85
Tunnel                           22
Pylône tubulaire                  3
Ouvrage d'art (pont, viaduc)      1
Pylône autostable                 1
Intérieur galerie                 1
Local technique                   1
Mât                               1
Name: NAT_LB_NOM, dtype: int64

'length: 737'




------------------
